In [3]:
from math import ceil
import time
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import matplotlib.image as mpimg

class Benchmark:
    def __init__(self, file_path, benchmark_type):
        self.file_path = file_path
        self.benchmark_type = benchmark_type
        self.subsets, self.universe_size, self.num_subsets = self.read_benchmark()
    
    def read_benchmark(self):
        """Lecture d'un benchmark MCP à partir d'un fichier texte"""
        with open(self.file_path, "r") as file:
            lines = file.readlines()
        
        m, n = map(int, lines[0].split())
        cost_lines_to_skip = ceil(n / 12) if self.benchmark_type == "4" else ceil(n / 15)
        subset_start_index = 1 + cost_lines_to_skip
        
        data_lines = lines[subset_start_index:]
        row_to_subsets = {}
        index = 0
        
        for row in range(1, m + 1):
            num_subsets = int(data_lines[index].strip())
            index += 1
            subsets = []
            while len(subsets) < num_subsets:
                subsets.extend(map(int, data_lines[index].split()))
                index += 1
            row_to_subsets[row] = subsets
        
        subset_to_rows = {}
        for row, subsets in row_to_subsets.items():
            for subset in subsets:
                if subset not in subset_to_rows:
                    subset_to_rows[subset] = []
                subset_to_rows[subset].append(row)
        return subset_to_rows, m, n

class DFSSolver:
    def __init__(self, benchmark, timeout=300):
        self.benchmark = benchmark
        self.k = ceil(benchmark.universe_size * 0.2) if benchmark.benchmark_type == "4" else ceil(benchmark.universe_size * 0.13)
        self.timeout = timeout
        self.start_time = None
        self.best_solution = []
        self.best_coverage = 0
        self.nodes_explored = 0
        self.timeout_occurred = False
        
        # Données pour la visualisation
        self.progress_data = {
            'time': [],
            'coverage': [],
            'nodes': [],
            'improvements': []
        }
        self.last_coverage = 0
    
    def solve(self):
        """Exécution principale avec tracking des données"""
        self.start_time = time.time()
        remaining = set(self.benchmark.subsets.keys())
        self.dfs(remaining, [], set())
        
        self.visualize_results()
        return {
            "k": self.k,
            "selected_subsets": self.best_solution,
            "coverage": self.best_coverage,
            "coverage_ratio": self.best_coverage / self.benchmark.universe_size,
            "nodes_explored": self.nodes_explored,
            "timeout_occurred": self.timeout_occurred,
            "time_taken": time.time() - self.start_time
        }
    
    def dfs(self, remaining, selected, covered):
        """DFS avec tracking des données"""
        if self.check_timeout():
            return
        
        self.nodes_explored += 1
        current_time = time.time() - self.start_time
        current_cov = len(covered)
        
        # Enregistrement des données de progression
        if current_cov > self.last_coverage:
            improvement = current_cov - self.last_coverage
            self.progress_data['time'].append(current_time)
            self.progress_data['coverage'].append(current_cov)
            self.progress_data['nodes'].append(self.nodes_explored)
            self.progress_data['improvements'].append(improvement)
            self.last_coverage = current_cov
            print(f"New best: {current_cov}/{self.benchmark.universe_size}")
        
        # Conditions d'arrêt
        if len(selected) == self.k or self.best_coverage == self.benchmark.universe_size:
            return
        
        # Élagage
        if len(selected) + len(remaining) < self.k:
            return
        
        upper_bound = current_cov + self.calculate_remaining_coverage(remaining, covered)
        if upper_bound <= self.best_coverage:
            return
        
        # Exploration
        for subset in sorted(remaining, 
                           key=lambda s: len(set(self.benchmark.subsets[s]) - covered),
                           reverse=True):
            if self.check_timeout():
                return
            new_covered = covered | set(self.benchmark.subsets[subset])
            self.dfs(remaining - {subset}, selected + [subset], new_covered)
    
    def calculate_remaining_coverage(self, remaining, covered):
        return len(set().union(*[self.benchmark.subsets[s] for s in remaining]) - covered)
    
    def check_timeout(self):
        if time.time() - self.start_time > self.timeout:
            self.timeout_occurred = True
            return True
        return False
    
    def visualize_results(self):
        """Crée des visualisations des performances"""
        plt.figure(figsize=(18, 12))
        
        # Graphique 1: Progression de la couverture
        plt.subplot(2, 2, 1)
        plt.plot(self.progress_data['time'], 
                self.progress_data['coverage'], 
                'b-', marker='o', markersize=4)
        plt.xlabel('Temps (s)')
        plt.ylabel('Éléments couverts')
        plt.title('Progression de la couverture')
        plt.grid(True)
        
        # Graphique 2: Efficacité de la recherche
        plt.subplot(2, 2, 2)
        plt.plot(self.progress_data['nodes'], 
                self.progress_data['coverage'], 
                'r-', marker='s', markersize=4)
        plt.xlabel('Noeuds explorés')
        plt.ylabel('Éléments couverts')
        plt.title('Efficacité de la recherche')
        plt.grid(True)
        
        # Graphique 3: Distribution des améliorations
        plt.subplot(2, 2, 3)
        if len(self.progress_data['improvements']) > 1:
            plt.hist(self.progress_data['improvements'], 
                    bins=min(20, len(set(self.progress_data['improvements']))), 
                    color='green', alpha=0.7)
            plt.xlabel('Amélioration par étape')
            plt.ylabel('Fréquence')
            plt.title('Distribution des améliorations')
            plt.grid(True)
        
        # Graphique 4: Vitesse d'exploration
        plt.subplot(2, 2, 4)
        if len(self.progress_data['time']) > 1:
            exploration_rate = np.diff(self.progress_data['nodes']) / np.diff(self.progress_data['time'])
            plt.plot(self.progress_data['time'][1:], 
                    exploration_rate, 
                    'm-', marker='^', markersize=4)
            plt.xlabel('Temps (s)')
            plt.ylabel('Noeuds/s')
            plt.title('Vitesse d\'exploration')
            plt.grid(True)
        
        plt.tight_layout()
        
        # Sauvegarde avec timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"dfs_progress_{timestamp}.png"
        plt.savefig(filename, dpi=300)
        print(f"\nVisualisation sauvegardée sous {filename}")
        plt.close()

if __name__ == "__main__":
    benchmark_file = "./Benchmark/4/scp41.txt"
    benchmark_type = "4"
    timeout = 60
    
    benchmark = Benchmark(benchmark_file, benchmark_type)
    print(f"Benchmark chargé: {benchmark.universe_size} éléments, {benchmark.num_subsets} sous-ensembles")
    print(f"k calculé: {ceil(benchmark.universe_size * 0.2)}")
    
    solver = DFSSolver(benchmark, timeout)
    result = solver.solve()
    
    print("\nRésultats finaux:")
    print(f"Sous-ensembles sélectionnés: {len(result['selected_subsets'])}/{result['k']}")
    print(f"Couverture: {result['coverage']}/{benchmark.universe_size} ({result['coverage_ratio']:.1%})")
    print(f"Noeuds explorés: {result['nodes_explored']}")
    print(f"Temps écoulé: {result['time_taken']:.2f}s")
    
    # Affichage des visualisations
    try:
        
        from glob import glob
        latest_file = max(glob('dfs_progress_*.png'), key=os.path.getctime)
        img = mpimg.imread(latest_file)
        plt.figure(figsize=(18, 12))
        plt.imshow(img)
        plt.axis('off')
        plt.show()
    except:
        print("Visualisation non disponible")

Benchmark chargé: 200 éléments, 1000 sous-ensembles
k calculé: 40
New best: 11/200
New best: 21/200
New best: 30/200
New best: 39/200
New best: 48/200
New best: 56/200
New best: 63/200
New best: 70/200
New best: 77/200
New best: 84/200
New best: 91/200
New best: 98/200
New best: 104/200
New best: 110/200
New best: 116/200
New best: 121/200
New best: 126/200
New best: 131/200
New best: 136/200
New best: 141/200
New best: 146/200
New best: 150/200
New best: 154/200
New best: 158/200
New best: 162/200
New best: 165/200
New best: 168/200
New best: 171/200
New best: 174/200
New best: 177/200
New best: 180/200
New best: 182/200
New best: 184/200
New best: 186/200
New best: 188/200
New best: 190/200
New best: 192/200
New best: 194/200
New best: 196/200
New best: 198/200


C:\Users\dell\AppData\Local\Temp\ipykernel_8096\2375073919.py:168: RuntimeWarning: divide by zero encountered in divide
  exploration_rate = np.diff(self.progress_data['nodes']) / np.diff(self.progress_data['time'])



Visualisation sauvegardée sous dfs_progress_20250403_103259.png

Résultats finaux:
Sous-ensembles sélectionnés: 0/40
Couverture: 0/200 (0.0%)
Noeuds explorés: 5772618
Temps écoulé: 61.85s
Visualisation non disponible
